In [2]:
import numpy as np
import pandas as pd
import re
import nltk

In [11]:
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/holy_cow/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
ps = PorterStemmer()
corpus = []

In [13]:
data = pd.read_csv('Restaurant_Reviews.tsv', delimiter='\t')
data.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [15]:
def textCleaner(review):
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    return review

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500)
data['Review'] = data['Review'].apply(textCleaner)

In [48]:
X = cv.fit_transform(data['Review']).toarray()
y = data.iloc[:, 1].values

In [49]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [56]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=500)
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [51]:
y_pred = rfc.predict(X_test)

In [52]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print(confusion_matrix(y_test, y_pred))

[[70 62]
 [23 95]]


In [53]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.75      0.53      0.62       132
          1       0.61      0.81      0.69       118

avg / total       0.68      0.66      0.65       250



In [54]:
print(accuracy_score(y_test, y_pred))

0.66
